# 2.8. 部署模型到生产环境中
## 🚄 前言
恭喜你！在经过模型微调和评测后，你的开发工作已经接近完成。本节课程将介绍最后一部分的工作：将你的模型部署在计算资源上，以调用微调后的模型，或提升模型的并发性能。

## 🍁 课程目标
学完本节课程后，你将能够：

➔ 了解为什么要部署模型；

➔ 掌握模型部署的基本方法；

➔ 了解将大模型应用集成到业务环境的方法；


## 📖 课程目录

- [1. 为什么要部署模型](#🤔-1-为什么要部署模型？)
    - [1.1. 你可能遇到的情况](#11-你可能遇到的情况)
- [2. 部署本地模型](#🚀-2-部署本地模型)
    - [2.1. 使用vLLM部署模型](#21-使用-vLLM-部署模型)
    - [2.2. 使用vLLM部署本地模型服务](#22-使用-vllm-部署本地模型服务)
    - [2.3. 在阿里云产品上部署模型](#23-在阿里云产品上部署模型)
- [3. 部署线上模型到百炼](#☁️-3-部署线上模型到百炼)
  - [3.1. 百炼模型部署服务](#31-百炼模型部署服务)
  - [3.2. 部署操作步骤](#32-部署操作步骤)
  - [3.3. 模型部署 API](#33-模型部署-api)
- [4. 拓展阅读：让机器人开始“上岗”](#🤖-4拓展阅读让机器人开始上岗)
- [5. 拓展阅读：提升部署效率的方法](#⚡-5-拓展阅读提升部署效率的方法)
   

## 🤔 1. 为什么要部署模型？

### 1.1. 你可能遇到的情况
想象一下，当你的新人答疑机器人第一次上线时，一切看起来都很顺利。可是，每天早上的忙碌时段，当几十位新员工同时使用机器人查询各类问题时，情况开始变得复杂。系统开始变慢，部分查询响应时间变长，甚至有些请求被直接丢弃。这种状况不仅影响了新员工的体验，还让管理层质疑机器人的可靠性。

以下是一个小实验，可以帮助你理解高负载下可能遇到的问题：

#### 小实验：从低负载到高负载理解机器人响应的成功率与平均响应时间
本实验旨在帮助你理解在不同峰谷时段（高并发和低并发）下，机器人响应的成功率和平均响应时间会如何变化。通过实际操作，您将直观地看到高并发请求可能导致调用失败或响应延迟，而低并发请求则更为稳定。
>⚠️ 运行实验代码可能会产生费用，请您在知悉费用详情后再进行实验。
> 
>截至2024年9月30日，示例中使用的通义千问2.5开源模型（Qwen2.5-0.5B-Instruct）处于限时免费状态，请您前往以下链接确认最新的模型价格。
>
>https://help.aliyun.com/zh/model-studio/getting-started/models


#### 使用 GPU 实例
使用 GPU 的主要目的是加速模型推理。在部署模型时，我们需要用到 vllm 加速库，这个库需要有 GPU 实例支持。

> 请注意：当学习[课时7：通过微调，提升模型的准确度与效率](https://edu.aliyun.com/course/3130200/lesson/343191255)或[课时11：部署模型到生产环境中](https://edu.aliyun.com/course/3130200/lesson/343260661)时，你需要申请一个 GPU 实例来完成课程。其他章节都可以仅使用 CPU 资源来完成。

请参考[课时2: 计算机环境准备](https://edu.aliyun.com/course/3130200/lesson/343181920?spm=a2cwt.28196072.ACP26.6.11f21094cuTqh1) 中的内容创建一个 GPU 实例。

#### ⚙️ 设置环境变量


安装依赖

In [3]:
! pip install -r requirements.txt
! pip install vllm==0.6.2

配置环境变量

In [ ]:
import os
import sys
sys.path.append("../")
from config.load_key import load_key
load_key()
print(os.environ["DASHSCOPE_API_KEY"])

##### 实验目标
- 理解峰谷时段：了解高并发（峰时）和低并发（谷时）对应用的影响。
- 测量成功率和响应时间：记录不同并发量下应用响应的成功率和平均响应时间。
- 识别应用的并发限制：通过实验确定应用在不同并发量下的表现，了解其最大承载能力。

##### 实验代码

以下代码定义了一个小实验，通过模拟不同并发量下通义千问模型调用API的响应，测量在峰谷时段下的调用成功率和平均响应时间。


In [6]:
import os
import asyncio
from openai import AsyncOpenAI
import platform
import time

# 创建 AsyncOpenAI 客户端
client = AsyncOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),  # 替换为您的API Key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # DashScope服务的base_url
)

# 定义异步请求的函数
async def make_request():
    try:
        start_time = time.time()
        completion = await client.chat.completions.create(
            model="qwen2.5-0.5b-instruct",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': '你是谁？'}
            ]
        )
        response_time = time.time() - start_time
        return (True, response_time)
    except Exception as e:
        return (False, str(e))

# 并发测试函数
async def test_concurrency(max_concurrent, duration, results):
    async def worker(worker_id):
        nonlocal results
        end_time = time.time() + duration
        while time.time() < end_time:
            success, result = await make_request()
            if success:
                results['success'] += 1
                results['total_time'] += result
            else:
                results['fail'] += 1
                print(f"Worker {worker_id} Failed: {result}")
            await asyncio.sleep(1)  # 每个worker每1秒发送一个请求

    # 创建并启动所有worker任务
    tasks = [asyncio.create_task(worker(i)) for i in range(max_concurrent)]
    await asyncio.gather(*tasks)

# 定义负载曲线，模拟峰谷时段的请求，每个步骤持续5秒
load_curve = [
    {'concurrent': 1, 'duration': 5},    
    {'concurrent': 2, 'duration': 5},   
    {'concurrent': 4, 'duration': 5},   
    {'concurrent': 2, 'duration': 5},   
    {'concurrent': 1, 'duration': 5},    
]

# 执行负载测试的主函数
async def main():
    print("开始负载测试...\n")
    for step in load_curve:
        concurrent = step['concurrent']
        duration = step['duration']
        results = {'success': 0, 'fail': 0, 'total_time': 0.0}
        print(f"测试并发量: {concurrent}，持续时间: {duration} 秒")
        start_time = time.time()
        await test_concurrency(concurrent, duration, results)
        actual_duration = time.time() - start_time
        total_requests = results['success'] + results['fail']
        avg_time = results['total_time'] / results['success'] if results['success'] > 0 else 0
        print(f"并发量: {concurrent}, 总请求: {total_requests}, 成功: {results['success']}, 失败: {results['fail']}, "
              f"平均响应时间: {avg_time:.2f}s, 实际持续时间: {actual_duration:.2f}s\n")
    print("负载测试完成。")

# 兼容Windows事件循环
if platform.system() == "Windows":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

# 运行主函数
if __name__ == "__main__":
    # Get the Tkinter event loop
    loop = asyncio.get_event_loop_policy().get_event_loop() 
    # Schedule the coroutine
    loop.create_task(main())  # Or loop.run_until_complete(my_async_function()) for immediate execution


##### 实验结果

执行这段代码后，你可能会看到如下输出：

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="90%">
<tbody>
<tr>
<td>

```bash
开始负载测试...

测试并发量: 1，持续时间: 5 秒
并发量: 1, 总请求: 3, 成功: 3, 失败: 0, 平均响应时间: 0.67s, 实际持续时间: 5.00s

测试并发量: 2，持续时间: 5 秒
并发量: 2, 总请求: 7, 成功: 7, 失败: 0, 平均响应时间: 0.98s, 实际持续时间: 7.53s

测试并发量: 4，持续时间: 5 秒
Worker 1 Failed: Error code: 429 - {'error': {'message': 'You exceeded your current requests list.', 'type': 'limit_requests', 'param': None, 'code': 'limit_requests'}}
并发量: 4, 总请求: 12, 成功: 11, 失败: 1, 平均响应时间: 1.73s, 实际持续时间: 9.22s

测试并发量: 2，持续时间: 5 秒
并发量: 2, 总请求: 5, 成功: 5, 失败: 0, 平均响应时间: 1.32s, 实际持续时间: 6.04s

测试并发量: 1，持续时间: 5 秒
并发量: 1, 总请求: 4, 成功: 4, 失败: 0, 平均响应时间: 0.61s, 实际持续时间: 6.45s
```
</td>
</tr>
</tbody>
</table>

##### 分析
你可以看到，当用户的总请求数逐渐增多后，大模型的平均响应时间不断增长。如果短时间的总请求数超过了模型服务的流量限制，用户的请求就会失败，这极大影响了用户的使用体验。

##### 总结

高并发应用常面临资源枯竭和系统崩溃的问题。当大量并发请求涌入时，服务器可能超负荷，导致用户体验下降甚至系统崩溃。这些问题在开发阶段难以察觉，但在实际环境中会被放大。
为了解决并发问题，你可以采用模型部署：

- 在负载高峰期增加计算资源，确保满足突发用户需求。
- 在负载低谷期减少计算资源，减少资源闲置导致的浪费。

具体方法将在本节的第三小节中详细介绍。

除了提高模型的并发量外，模型部署常被用来加载微调后的模型，并且对外提供调用服务。接下来，你将从部署微调后的模型开始，学习部署模型的基本方法。


## 🚀 2. 部署本地模型

让我们先从你的新人答疑机器人开始入手。在2.4课程里，你已经通过微调改善了模型性能。那么，如何让微调后的模型发挥作用呢？模型部署就是这个问题的答案。在这部分课程中，你将学习如何使用vLLM快速部署微调模型，以便供你的机器人使用。

### 2.1. 使用 vLLM 部署模型
vLLM 是一种高性能的语言模型推理引擎，专为优化大语言模型（LLM）的运行效率而设计。vLLM 的核心目标是通过改进缓存策略、优化内存管理、提升计算资源利用率，显著加速语言模型的推理速度。它适用于在生产环境中部署大语言模型，尤其是需要高并发处理能力和低延迟的应用场景。

vLLM 具有以下特点，你可以简单了解一下：

1. 高效的并发管理：vLLM 能够同时处理大量的推理请求，支持高并发的推理任务，极大地提高了吞吐量，特别适用于对话式AI、实时问答系统等场景。
2. 内存优化与碎片管理：vLLM 通过先进的内存分配和释放机制，减少内存碎片，提高内存利用率，显著降低运行大模型时的内存占用。
3. 改进的缓存策略：vLLM 引入了创新的缓存技术，能够高效地重用推理过程中生成的中间计算结果，从而减少不必要的重复计算，提升模型运行速度。
4. 灵活的模型部署：vLLM 支持多种语言模型的部署，包括基础模型和经过微调的模型（如 safetensors 格式的模型），能够适应不同的应用需求。

总之，使用 vLLM 部署模型是一个简单且快速的方式，让我们一起看看如何使用vLLM部署你的微调模型。


### 2.2. 使用 vLLM 部署本地模型服务
接下来，你将学习如何使用 ModelScope Notebook 提供的免费 GPU 实例环境，使用 vLLM 部署一个本地的模型服务。通过实际操作，你将使用 vLLM 部署微调模型，以提供 OpenAI API 兼容的推理服务。

#### 2.2.1. 安装与环境准备
这部分实验需要使用GPU环境完成，你可以按照第一节课程：环境准备的方法，使用PAI-DSW提供的免费GPU实例。此外，你还需要将微调好的模型文件目录上传到新的GPU实例中，方便后续进行部署。

<div align="center">
    <img src="https://gw.alicdn.com/imgextra/i1/O1CN01D7jaPO1nLUF4t5op4_!!6000000005073-0-tps-2038-740.jpg" alt="问答流程" width="650"/>
</div>

vLLM所需的运行环境，相信你已经在第一节课安装好了，如果在后续的操作中遇到了任何问题，你也可以按照下列操作尝试修复：

>安装 vLLM：在终端中安装 vLLM 及其相关依赖：pip install vllm==0.6.2

#### 2.2.2. 部署模型为 OpenAI 兼容服务
**启动服务**：vLLM 提供了一个 OpenAI 兼容的服务器模式，能够直接部署微调模型。例如，使用以下命令来启动服务：其中，path/to/your-model/为模型数据和配置文件的存放目录。
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="90%">
<tbody>
<tr>
<td>

```bash
vllm serve "/path/to/your-model/" --load-format "safetensors" --port 8000 --dtype=half
```
</td>
</tr>
</tbody>
</table>
注意，如果你遇到了接口占用错误，你也可以修改默认端口为8001、8002……（0～65535之间的任意值，且没有被占用）。
配置 Docker 部署（可选）：这种方法仅供参考。你也可以使用 vLLM 提供的官方 Docker 镜像进行部署，确保你拥有合适的 GPU 驱动和共享内存配置：此 Docker 镜像可以将模型部署为一个 OpenAI API 兼容的服务，同时利用 vLLM 的高效推理能力 

<table width="90%">
<tbody>
<tr>
<td>

```bash
docker run --runtime nvidia --gpus all \
    -v ~/.cache/huggingface:/root/.cache/huggingface \
    -p 8000:8000 \
    vllm/vllm-openai:latest \
    --model path/to/your-model/
```
</td>
</tr>
</tbody>
</table>


#### 2.2.3. 测试部署效果
**发送测试请求**：通过 ```curl``` 或 ```Postman``` 等工具，发送测试请求来验证部署是否成功。例如，可以使用以下 ```curl``` 命令测试模型推理效果：通过该命令，你可以验证模型是否正确处理并返回 OpenAI API 兼容的响应格式 
<table width="90%">
<tbody>
<tr>
<td>

```bash
curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "/path/to/your/model/",
        "messages": [
            {"role": "system", "content": "你是一个帮助用户的助手。"},
            {"role": "user", "content": "请告诉我2008年北京奥运会，中国队获得了多少枚金牌？"}
        ]
    }'
```
</td>
</tr>
</tbody>
</table>
如果你看到了模型回复的内容，恭喜你，你已经完成了微调模型的任务！后续你还可以将这个测试请求集成到机器人里，让微调模型为你的机器人生产内容。

你可能会问：学习完所有课程后，我想沿用这种部署方法，但部署环境会发生变化，例如本地服务器、云服务器、甚至是其他云应用。那么，部署的方法会不会改变呢？

接下来，我们将以阿里云的三种产品为例解答你的问题。你将学习硬件资源的选择方法；ECS、PAI、FC三种产品之间的区别，以及如何在这些产品上部署模型。


### 2.3. 在阿里云产品上部署模型

在开始部署模型之前，你需要规划好硬件资源的使用计划。不同的模型对计算资源有不同的需求，特别是在GPU显存和计算能力方面。

#### 2.3.1. 硬件资源与模型参数量

在部署模型时，必须考虑模型的参数量和计算资源需求。参数量越大，对GPU的显存和计算能力要求越高。以下是常见的开源模型如Qwen、Llama等在不同参数量级别下的GPU资源需求：

- 小型模型（7B参数以下）：
    - 可以使用单个GPU进行部署。
    - 适用GPU示例：NVIDIA T4、V100，16GB VRAM。
- 中型模型（7B-13B参数）：
    - 建议使用 VRAM 较大的 GPU 以满足模型的需求（超过32 GB）
    - 适用GPU示例：NVIDIA V100 32GB、A100 40GB。
- 大型模型（30B参数以上）：
    - 需要多GPU进行分布式部署，或者使用显存非常高的GPU。
    - 适用GPU示例：NVIDIA A100 80GB。
  
以通义千问2.5开源模型为例（Qwen2.5-7B-Instruct），我们建议你使用不低于16GB VRAM 的硬件资源。由于硬件资源有限，你可能希望在尽可能少的资源上部署更大的模型，在拓展阅读部分，我们将介绍一些提升模型部署的方法，希望对你有所启发。

接下来，你将了解三种阿里云产品之间的对比，帮助你选择“量身定制”的部署环境。
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
#### 2.3.2. 阿里云产品对比

以下是阿里云产品（云服务器ECS、人工智能平台PAI、函数计算FC）的对比表格，你可以选取最关心的几个维度进行比较，其他的内容只需了解即可。

|  **对比维度**  |  **ECS (Elastic Compute Service)**  |  **PAI (Platform for AI)**  |  **FC (Function Compute)**  |
| --- | --- | --- | --- |
|  **时间（部署复杂性）**  |  复杂，需深入理解配置，适合有经验的用户  |  较简单，通过图形化界面操作，自动化程度高  |  最简单，无需管理底层服务器，部署速度快  |
|  **成本**  |  较低，长期使用可通过预留实例优化成本  |  自动扩展功能优化成本，但总体成本略高  |  按需计费，轻量任务成本低  |
|  **稳定性**  |  高，适合长期、稳定运行的服务  |  高，支持大规模并发和高效管理  |  高，适合短时、高并发任务  |
|  **性能与扩展性**  |  提供全面控制，易于水平和垂直扩展  |  自动化扩展，适合大规模推理任务  |  自动伸缩，但对高性能任务支持有限  |
|  **开发和部署的复杂性**  |  复杂但灵活，适合有经验的团队  |  较简单，适合快速上线  |  最简便，适合快速开发和测试  |
|  **安全与合规性**  |  高，支持 RAM、VPC 隔离等高级安全功能  |  中等，支持版本控制和访问管理  |  基础安全配置，适合低要求任务  |
|  **运维管理与自动化**  |  需手动运维，运维工作量较大  |  自动化管理，减少运维负担  |  最轻，基础设施管理由平台负责  |
|  **集成与生态兼容性**  |  兼容性好，与其他云服务无缝集成  |  与 AI 工具链深度集成，支持 CI/CD  |  适合无服务器架构，易与其他无服务器服务集成  |

接下来，我们将分别介绍每个产品的模型部署方法。这些方法可能与最初版本有些不同，不过你也不用担心，阿里云产品可以快速地完成模型部署任务。

#### 2.3.3. 使用vLLM部署模型到阿里云 ECS

相比在ModelScope Notebook上部署模型，在云服务器ECS中，你可能需要了解一些基本的软硬件知识，我们已经为你提前准备了一些：

##### 部署知识

在阿里云 ECS 上部署 vLLM 时，系统架构主要由以下部分组成：

*   ECS 实例：提供 GPU 计算能力支持大语言模型的推理。
    
*   vLLM 推理引擎：安装在 ECS 实例上，负责加载和执行语言模型的推理任务。
    
*   容器管理与服务：可使用 Docker 来管理模型部署，确保推理服务稳定运行。
    
*   监控与日志系统：通过阿里云云监控服务（如 CloudMonitor），实时监控 GPU 实例的性能与运行状态，及时发现并处理异常。
    

##### 部署建议

以下是一些部署建议，仅供参考：

- 机器选择
    -  根据模型的参数量和计算需求，选择适配 GPU 的实例类型。
    -  常用的 GPU 如 NVIDIA Tesla T4，适合中小型模型部署需求。
    -  对于大型模型和高性能需求，推荐使用 NVIDIA A100。  
- 成本考虑
    - GPU 实例的成本较高，需根据实际的业务负载选择实例规格。
    - 建议在非高峰期调整 GPU 实例数，以减少计算成本，避免长期高占用的固定成本。
- 端口配置
    - 在阿里云云控制台中，配置 ECS 安全组规则。
    - 开放模型服务使用的端口（如 5000），确保外部用户可以正常访问。  
- 白名单 IP
    - 通过安全组设置白名单 IP，限制仅授权用户访问服务，增强安全性。
    

##### 部署方法

在云服务器ECS上，我们推荐你使用Docker及vLLM快速部署模型。由于篇幅限制，你可以通过ECS官方文档了解部署方法。

- [使用vLLM容器镜像快速构建大预言模型在GPU上的推理环境](https://help.aliyun.com/zh/egs/use-cases/use-a-vllm-container-image-to-run-inference-tasks-on-a-gpu-accelerated-instance)

#### 2.3.4. 部署到阿里云 PAI - EAS

在阿里云人工智能平台 PAI，你可以通过模型在线服务 EAS 快速部署模型。我们为你准备了一些前置知识，帮助你快速了解产品。

##### 前置知识

在阿里云 PAI-EAS 平台上使用 vLLM 部署大语言模型时，系统架构包括以下主要部分：

- EAS 服务实例：使用 EAS（Elastic Algorithm Service）作为模型推理平台，管理和提供高并发、低延迟的模型服务。
    
- vLLM 推理引擎：vLLM 提供优化的推理引擎，用于高效地运行大语言模型，支持并发管理和内存优化。
    
- 模型与数据存储：通过阿里云的 OSS（对象存储）或 NAS，将模型权重和代码挂载到 EAS 实例，确保数据访问和更新的便利性。
    
- Web 接口与 API 服务：部署完成后，EAS 生成的 Web 应用或 API 服务提供用户交互，支持实时请求和响应处理。
    
- 监控与日志系统：使用 PAI 的监控功能，对模型服务的性能进行实时监控和优化。
    

##### 部署建议

以下是一些部署建议，仅供参考：

- 资源配置
    - EAS 支持多种 GPU 实例，常见选择包括 ml.g7i.c16m60.1-gpu30，适用于对性能要求较高的推理任务。
    - 可根据业务需求选择适合的实例类型，同时启用推理加速功能（如 BladeLLM、vLLM）以提升性能。
- 成本控制
    - 建议在负载较高时使用专用资源组，并在低负载时自动缩减资源以控制成本。
    - 使用包月方式购买可降低长时间运行的费用。
- 端口与网络安全
    - EAS 自动配置推理服务的端口，可通过阿里云控制台调整网络安全组规则，限制对服务的访问权限。
        

##### 部署方法

要在PAI-EAS上使用vLLM部署，您可以遵循以下步骤：

1.  **选择部署方式**：
    
    1.  登录PAI控制台，进入相应的工作空间。
        
    2.  导航至“模型部署 > 模型在线服务（EAS）”页面，点击“部署服务”。
        
2.  **自定义模型部署**：
    

在“自定义模型部署”区域，点击“自定义部署”，进入“新建服务”页面。

3.  **配置服务信息**：
    
    1.  服务名称：输入自定义的模型服务名称。
        
    2.  部署方式：选择“镜像部署AI-Web应用”。
        
4.  **选择镜像**：
    

镜像选择：在PAI平台镜像列表中，选择“chat-llm-webui”，确保镜像版本选择为支持vLLM的版本，如“3.0-vllm”。

5.  **模型配置**（如需挂载自定义模型）：
    
    1.  点击“填写模型配置”，配置OSS挂载路径，例如模型文件所在OSS路径为`oss://bucket-name/path/to/model`。
        
    2.  挂载路径设置为`/data`，确保与模型配置中的挂载路径一致。
        
    3.  是否只读开关通常关闭，除非有特殊需求。
        
6.  **推理加速配置**：
    

在资源配置区域，找到“推理加速”参数，选择“开源框架vllm推理加速”。

7.  **完成配置并部署**：
    

审核所有配置信息无误后，点击“部署”按钮启动部署流程。

通过以上步骤，您就可以在PAI-EAS平台上利用vLLM推理加速引擎部署大语言模型了。请注意，使用vLLM推理加速时，EAS服务将不支持LangChain功能。此外，确保模型配置与所选推理加速引擎兼容，以达到最佳性能效果。

**参考链接**

- [LLM大语言模型 部署EAS服务](https://help.aliyun.com/zh/pai/user-guide/deploy-an-llm/) 

- [5分钟使用EAS一键部署LLM大语言模型应用 常见问题及解决方法](https://help.aliyun.com/zh/pai/use-cases/deploy-llm-in-eas)

- [LLM智能路由：提升LLM推理系统整体效率 部署服务](https://help.aliyun.com/zh/pai/user-guide/use-llm-intelligent-router-to-improve-inference-efficiency)

#### 2.3.5. 部署到阿里云 FC

函数计算 FC 提供了一种更加轻量的部署环境，我们也为你准备了一些前置知识，帮助你快速了解产品。

##### 前置知识

在 FC 平台上使用本地模型时，系统架构主要由以下几个部分组成：

- 函数计算实例：作为主要的执行环境，负责处理所有的模型推理请求。FC 支持事件驱动的架构，可以灵活地根据请求量自动扩展或缩容实例数量。
    
- 模型存储与加载：使用阿里云 OSS（对象存储服务）或直接将模型文件打包到 Docker 镜像中，并在函数启动时加载。这种方法可以有效减少加载延迟并简化模型更新流程。
    
- 容器与运行时管理：可以通过自定义 Docker 镜像来定制模型推理所需的依赖环境，并将该镜像部署到 FC 实例中进行高效的模型推理。
    
- 触发器与 API 服务：通过 HTTP、定时器、事件等触发器启动模型推理服务，支持灵活的调用方式和实时的响应处理。
    

##### 部署建议

以下是一些部署建议，仅供参考：

*   实例选择
    
    *   FC 支持多种实例类型，推荐使用 GPU 加速实例来提升大模型的推理性能。
        
    *   实例类型和配置可在创建服务时指定，根据实际业务需求选择合适的 CPU、内存和 GPU 资源。
        
*   成本控制
    
    *   函数计算采用按需计费模式，可以在高峰期自动扩展实例数量满足请求，同时在低峰期缩减实例数，避免资源浪费。
        
    *   针对 GPU 实例的长时间任务，可以通过预置实例来优化启动时间和降低成本。
        
*   端口与安全配置
    
    *   在 FC 控制台中配置服务的触发器和安全组，确保只开放必要的端口并设置 IP 白名单来防止未授权的访问。
        

#### 部署方法

要在函数计算的GPU实例上部署模型，您可以遵循以下步骤：

1.  **创建函数应用**：
    
    登录阿里云[函数计算控制台](https://fcnext.console.aliyun.com/)，创建一个新的函数应用。选择合适的运行时环境（如Python），并配置相应的函数计算资源，包括实例规格、内存、存储等。
        
2.  **上传或选择镜像**：
    
    如果您已经拥有vLLM的Docker镜像，可以直接在函数计算中上传该镜像；或者，如果镜像已托管在阿里云容器镜像服务或其他支持的镜像仓库中，直接引用该镜像地址。
        
3.  **配置函数**：
    
    在函数配置中，指定入口点（如主函数），并设置环境变量，包括模型路径、端口等，确保函数能够正确加载模型和启动服务。
        
4.  **部署并检查**：
    
    部署函数后，检查函数的 **镜像加速准备状态** ，确保其变为 **可用** 。这是函数计算针对大型AI镜像提供的优化功能，有助于缩短启动时间。
        
5.  **测试与验证**：
    
    通过访问函数计算提供的访问域名或API Gateway，发送测试请求以验证vLLM服务是否正常运行，能否正确响应推理请求。
        

**参考链接**

- [实时推理场景基本信息](https://help.aliyun.com/zh/functioncompute/fc-2-0/use-cases/real-time-inference-scenarios)

- [准实时推理场景的容器支持](https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/quasi-real-time-inference-scenarios)

- [实时推理场景的部署方式](https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/real-time-inference-scenarios-1)

到目前为止，我们介绍的模型部署方法都是基于本地的模型文件，无论是开源模型还是微调模型。接下来我们将介绍闭源或线上模型的部署方法。由于这一类模型的特点，不同模型厂商提供的模型部署服务存在差异。因此，我们将从阿里云百炼提供的模型部署服务入手，向您介绍通义千问线上模型的部署方法。

## ☁️ 3. 部署线上模型到百炼

在本节课程的开始，你已经通过实验了解了模型部署的意义。无论是在实验中，还是在你的机器人项目上，都使用了通义千问线上模型。那么，如何通过部署来扩充这些线上模型的并发量呢？

阿里云百炼为这类模型开发了简明的部署服务，你无需关注模型部署的具体实践，也能快速创建部署任务。

### 3.1. 百炼模型部署服务

百炼模型部署服务旨在解决大模型在应对高并发请求时的性能问题。通过独占实例的灵活管理，用户可以动态扩展或缩减模型的并发处理能力，从而保证在流量高峰期也能高效响应用户请求，确保系统稳定运行。

百炼模型部署服务具备以下功能：

1.  **创建部署服务**：通过API创建一个新的模型部署任务，指定模型名称和独占实例数量。
    
2.  **查询部署状态**：检查部署任务的状态，确认服务是否已成功运行。
    
3.  **更新部署服务**：根据需求动态调整独占实例的数量，增加或减少并发处理能力。
    
4.  **删除部署服务**：当不再需要某个部署服务时，通过API删除任务以释放资源。
    

### 3.2. 部署方法

百炼模型部署服务提供了一套完备的部署方案，使你可以方便、灵活地控制部署的资源量。你可以参考以下文档：

- [大模型服务平台百炼的模型部署](https://help.aliyun.com/zh/model-studio/user-guide/model-deployment)


## 🤖 4. 拓展阅读：让机器人开始“上岗”

在经过部署优化后，新人答疑机器人具备了更强的并发处理能力。接下来，我们将提供两个简单的发布方式，方便你了解发布机器人的基本流程：

### 4.1. 将机器人集成为网站 AI 助手

在阿里云百炼文档中，我们提供了一个快速启动的示例，帮助千万用户快速发布百炼RAG应用到他们自己的网站上，作为 AI 助手来服务客户。现在，你也可以参照这篇最佳实践文档，将本课程中开发的机器人也发布为网站的AI助手，以一种更便捷的形式为新员工提供服务。

- [10分钟在网站上增加一个AI助手](https://help.aliyun.com/zh/model-studio/use-cases/add-an-ai-assistant-to-your-website-in-10-minutes)

### 4.2. 创建钉钉群AI机器人

我们也提供了“百炼应用发布为钉钉群AI机器人的方法”示例，你也可以参照这篇文档来发布属于自己的钉钉群机器人，让新员工即刻获取回答。

- [10分钟在钉钉上增加一个AI机器人](https://help.aliyun.com/zh/model-studio/use-cases/add-an-ai-assistant-to-your-dingtalk-in-10-minutes)

### 4.3. 小结

无论是选择发布到网站AI助手还是钉钉机器人，甚至是其他形式的AI应用，你将成功让新人答疑机器人的“上岗”。这将极大提高新员工的使用便利性，并有效降低答疑成本。

## ⚡ 5. 拓展阅读：提升部署效率的方法

在部署本地模型时，你可能会遇到硬件资源不足的问题，以下是一些提升部署效率的方法。

### 5.1. 量化（Quantization）

你的服务器显存和存储空间不足以支持大模型的部署任务。通过将模型权重和激活值从32位浮点数转换为8位整数，你大幅减少了内存占用和计算量。模型现在不仅能运行起来，还能在推理时显著加速，适用于你在本地部署实时AI应用的需求。

-  **概念**：量化是将模型权重和激活值从高精度（如32位浮点数）转换为低精度（如8位整数）表示的一种技术。量化可以显著减少模型的存储和计算资源需求，同时在推理过程中提高速度和节省内存。
    
-  **优势**：
    
    *   **计算加速**：低精度计算通常比高精度计算更快，尤其是在硬件（如GPU、TPU）上有专门的优化。
        
    *   **资源节省**：降低内存使用和存储空间，使模型更适合部署在资源受限的设备上（如移动设备和嵌入式系统）。
        

### 5.2. 蒸馏（Distillation）

你当前的服务器无法承载完整的大模型。于是，你训练了一个较小的学生模型来模仿大模型的行为。虽然学生模型比原来的大模型小得多，但通过知识蒸馏，它保持了接近大模型的性能，从而能够在你的有限硬件上高效运行。

*   **概念**：模型蒸馏是一种通过训练一个较小的学生模型来模仿较大教师模型的方法。教师模型通常是一个性能优越但复杂度较高的大模型，而学生模型则较小且简化，但通过蒸馏，它能够接近教师模型的性能。
    
*   **优势**：
    
    *   **模型压缩**：减少模型的参数和计算复杂度，使其更轻量化。
        
    *   **推理加速**：学生模型通常比教师模型更快，适合在推理阶段使用。
        

### 5.3. 缓存（Caching）

由于部分推理请求是重复的，你通过缓存常见的推理结果，避免了对模型的重复计算。每当有相似的请求时，系统直接从缓存中返回结果，大大减少了计算资源的消耗，使得整个系统更加高效。

*   **概念**：缓存是一种通过存储计算结果来减少重复计算、提高响应效率的技术。对于一些常见的请求结果，可以提前计算并存储在缓存中，以便快速返回结果。
    
*   **优势**：
    
    *   **减少重复计算**：提高系统的响应速度和效率。
        
    *   **降低资源消耗**：减少计算资源的使用，优化系统性能。
        

### 5.4. 负载均衡（Load Balancing）

你将部署的vLLM应用扩展到多个服务器实例，并使用负载均衡器将用户请求均匀分配给各个实例。这不仅防止某个实例过载，还确保即使某个服务器出现故障，其他实例也能无缝接管工作，保证模型的高可用性。

*   **概念**：负载均衡是一种通过将网络请求均匀分配到多个服务器实例上，从而提高系统的稳定性和响应速度的技术。
    
*   **优势**：
    
    *   **避免单点故障**：提高系统的可靠性和可用性。
        
    *   **优化资源使用**：均衡负载，避免某个实例过载。
        

### 5.5.数据并行和模型并行

*   **数据并行（Data Parallelism）**：  
    为了加快推理，你将大数据集切分为多个小批次，并行在多个服务器上运行相同的模型。这样即使你的硬件配置不高，整个训练任务也能并行化运行，快速完成。
    
    *   **概念**：将数据集划分成多个小批次，并行地在多个计算节点上处理相同的模型。这种方法适用于推理的过程，能够显著加快计算速度。
        
    *   **优势**：
        
        *   **加速计算**：通过并行处理多个数据批次，提高整体计算速度。
            
        *   **负载均衡**：均匀分配计算任务，避免单个节点的过载。
            
*   **模型并行（Model Parallelism）**：  
    你的模型太大，无法单独在一个服务器上运行。于是，你将模型的不同层分配到不同的节点上运行。这样多个服务器协同处理，使得大模型能够运行在多个设备上，充分利用每个节点的计算资源。
    
    *   **概念**：将模型的不同部分（如不同层或模块）分布在不同的计算节点上进行处理。这种方法适用于超大模型的部署，能够有效利用多个计算资源。
        
    *   **优势**：
        
        *   **处理大模型**：适用于超大模型，使其能够在多个节点上协同运行。
            
        *   **资源优化**：通过分布式计算，充分利用各节点的计算能力。
            

## ✅本节小结

本节课程详细介绍了模型部署的目的和基本方法，你学习了：
- 模型部署的意义，以及部署微调模型的方法。
- 部署本地模型的方法，包括微调模型和开源模型，以提供模型推理服务。
- 部署线上模型的方法，以提高模型的并发处理能力。
- 将机器人集成到业务环境的方法，以及优化部署效率的方法。

通过本节的学习，你已经掌握了模型部署的基本方法，为构建高性能、可扩展的大模型应用打下了坚实的基础。

接下来，你将学习一些借助大模型辅助内容生产的案例，如翻译、音视频转换、PPT制作等。希望这些案例能启发你开发更多效率工具。

